There are methods that are common in machine learning projects like cross validation, grid searching and stacking.

So I wrapped them into a common module in helper.py. 

In this script, I will call those functions directly. Feel free to have a check at my github for source code.



In [21]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, BayesianRidge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from xgboost import XGBRegressor

import numpy as np
import pandas as pd


Here is a tip for building your own module. 

During training, it's likely to do some modifications on the module file. We need to reload the module every time we have modified it.

Just as following ("helpers" is the name of module I built):

In [22]:
import helpers
from importlib import reload
reload(helpers)
from helpers import *

## Load processed data

I have documented details of my data processing pipeline. Feel free to have a check on my blog.

In [23]:
# X = np.load("X.npy")
# y = np.load("y.npy")
# scaled_test = np.load("scaled_test.npy")

X = np.load("X_scaled.npy")
y = np.load("y_log.npy")
scaled_test = np.load("test_X_scaled.npy")

In [24]:
X.shape

(1458, 410)

In [25]:
models = [LinearRegression(),Ridge(),Lasso(alpha=0.01,max_iter=10000),RandomForestRegressor(),GradientBoostingRegressor(),SVR(),LinearSVR(),
          ElasticNet(alpha=0.001,max_iter=10000),SGDRegressor(max_iter=1000,tol=1e-3),BayesianRidge(),KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5),
          ExtraTreesRegressor(),XGBRegressor()]

names = ["LR", "Ridge", "Lasso", "RF", "GBR", "SVR", "LinSVR", "Ela","SGD","Bay","Ker","Extra","Xgb"]
for name, model in zip(names, models):
    score = rmse_cv(model, X, y)
    print("{}: {:.6f}, {:.4f}".format(name,score.mean(),score.std()))



LR: 1860147728.348905, 1380448048.8361
Ridge: 0.117598, 0.0091
Lasso: 0.121594, 0.0058
RF: 0.143567, 0.0047
GBR: 0.125229, 0.0070
SVR: 0.112757, 0.0048
LinSVR: 0.122644, 0.0083
Ela: 0.111220, 0.0062
SGD: 0.163672, 0.0121
Bay: 0.110566, 0.0060
Ker: 0.109279, 0.0055
Extra: 0.139001, 0.0061
Xgb: 0.125494, 0.0063


In [26]:
lasso_best = grid(Lasso()).grid_get(X,y,{'alpha': [0.0004,0.0005,0.0007,0.0006,0.0009,0.0008],'max_iter':[10000]})

{'max_iter': 10000, 'alpha': 0.0005} 0.10870999058248038
                                 params  mean_test_score  std_test_score
0  {'max_iter': 10000, 'alpha': 0.0004}         0.108710        0.003045
1  {'max_iter': 10000, 'alpha': 0.0005}         0.108710        0.003020
2  {'max_iter': 10000, 'alpha': 0.0007}         0.109444        0.003036
3  {'max_iter': 10000, 'alpha': 0.0006}         0.109015        0.003012
4  {'max_iter': 10000, 'alpha': 0.0009}         0.110222        0.003026
5  {'max_iter': 10000, 'alpha': 0.0008}         0.109866        0.003050


In [27]:
ridge_best=grid(Ridge()).grid_get(X,y,{'alpha':[35,40,45,50,55,60,65,70,80,90]})

{'alpha': 35} 0.10863738614860778
          params  mean_test_score  std_test_score
0  {'alpha': 35}         0.108637        0.003011
1  {'alpha': 40}         0.108655        0.002995
2  {'alpha': 45}         0.108684        0.002982
3  {'alpha': 50}         0.108719        0.002972
4  {'alpha': 55}         0.108761        0.002964
5  {'alpha': 60}         0.108806        0.002958
6  {'alpha': 65}         0.108855        0.002953
7  {'alpha': 70}         0.108906        0.002950
8  {'alpha': 80}         0.109014        0.002945
9  {'alpha': 90}         0.109128        0.002943


In [28]:
svr_best = grid(SVR()).grid_get(X,y,{'C':[11,12,13,14,15],'kernel':["rbf"],"gamma":[0.0003,0.0004],"epsilon":[0.008,0.009]})


{'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': 0.008, 'C': 15} 0.10613736447556428
                                               params  mean_test_score  \
0   {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106940   
1   {'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': ...         0.106369   
2   {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106881   
3   {'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': ...         0.106355   
4   {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106825   
5   {'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': ...         0.106260   
6   {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106781   
7   {'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': ...         0.106246   
8   {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106715   
9   {'gamma': 0.0004, 'kernel': 'rbf', 'epsilon': ...         0.106181   
10  {'gamma': 0.0003, 'kernel': 'rbf', 'epsilon': ...         0.106698   
11  {'gamma': 0.0004, 'kernel'

In [29]:
param_grid={'alpha':[0.2,0.3,0.4,0.5], 'kernel':["polynomial"], 'degree':[3],'coef0':[0.8,1,1.2]}
ker_best = grid(KernelRidge()).grid_get(X,y,param_grid)



{'kernel': 'polynomial', 'degree': 3, 'coef0': 1.2, 'alpha': 0.3} 0.1066923330086768
                                               params  mean_test_score  \
0   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106743   
1   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106712   
2   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106905   
3   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.107054   
4   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106698   
5   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106692   
6   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.107520   
7   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106868   
8   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.106712   
9   {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.108056   
10  {'kernel': 'polynomial', 'degree': 3, 'coef0':...         0.107108   
11  {'kernel': 'polynomial'

In [30]:
ela_best = grid(ElasticNet()).grid_get(X,y,{'alpha':[0.0005,0.0008,0.004,0.005],'l1_ratio':[0.08,0.1,0.3,0.5,0.7],'max_iter':[10000]})

{'max_iter': 10000, 'l1_ratio': 0.5, 'alpha': 0.0008} 0.10870722583030155
                                               params  mean_test_score  \
0   {'max_iter': 10000, 'l1_ratio': 0.08, 'alpha':...         0.112507   
1   {'max_iter': 10000, 'l1_ratio': 0.1, 'alpha': ...         0.112061   
2   {'max_iter': 10000, 'l1_ratio': 0.3, 'alpha': ...         0.109899   
3   {'max_iter': 10000, 'l1_ratio': 0.5, 'alpha': ...         0.109294   
4   {'max_iter': 10000, 'l1_ratio': 0.7, 'alpha': ...         0.108820   
5   {'max_iter': 10000, 'l1_ratio': 0.08, 'alpha':...         0.111120   
6   {'max_iter': 10000, 'l1_ratio': 0.1, 'alpha': ...         0.110722   
7   {'max_iter': 10000, 'l1_ratio': 0.3, 'alpha': ...         0.109269   
8   {'max_iter': 10000, 'l1_ratio': 0.5, 'alpha': ...         0.108707   
9   {'max_iter': 10000, 'l1_ratio': 0.7, 'alpha': ...         0.108849   
10  {'max_iter': 10000, 'l1_ratio': 0.08, 'alpha':...         0.108820   
11  {'max_iter': 10000, 'l1_ratio': 0.

In [31]:
bay = BayesianRidge()

## Stacking

In [32]:
lasso = Lasso(alpha=0.0005,max_iter=10000)
ridge = Ridge(alpha=60)
svr = SVR(gamma= 0.0004,kernel='rbf',C=13,epsilon=0.009)
ker = KernelRidge(alpha=0.2 ,kernel='polynomial',degree=3 , coef0=0.8)
ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
bay = BayesianRidge()

In [33]:
# stack_model = stacking(mod=[lasso_best,ridge_best,svr_best,ker_best,ela_best,bay],meta_model=ker_best)
stack_model = stacking(mod=[lasso,ridge,svr,ker,ela,bay],meta_model=ker)
print(rmse_cv(stack_model,X,y))


[0.10298996 0.10928527 0.1170537  0.09877946 0.10450595]


In [34]:
X_train_stack, X_test_stack = stack_model.get_oof(X,y,scaled_test)
X_train_add = np.hstack((X,X_train_stack))
X_test_add = np.hstack((scaled_test,X_test_stack))
print(rmse_cv(stack_model,X_train_add,y))



[0.09843626 0.10439415 0.11256335 0.09446302 0.09983553]


## Finally!

In [35]:
stack_model.fit(X,y)

stacking(meta_model=KernelRidge(alpha=0.2, coef0=0.8, degree=3, gamma=None, kernel='polynomial',
      kernel_params=None),
     mod=[Lasso(alpha=0.0005, copy_X=True, fit_intercept=True, max_iter=10000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False), Ridge(alpha=60, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=No...True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)])

In [36]:
pred = np.exp(stack_model.predict(scaled_test))

In [37]:
sub=pd.read_csv("sample_submission.csv")
sub.tail(1)

,Id,SalePrice
1458,2919,187741.866657


In [38]:
from time import gmtime, strftime
submName = strftime("%Y%m%d%H%M%S", gmtime()) + '_submission.csv'
sub['SalePrice']=pred
sub.to_csv(submName, index=False)